# Data Import & Cleaning

In [1]:
# import dependencies
import requests
import json
import pprint
import pandas as pd
from api_keys import api_key

In [4]:
# api call
url = f"https://api.waqi.info/v2/map/bounds?latlng=-90,-180,90,180&networks=all&token={api_key}"
response = requests.get(url)
data = response.json()

In [5]:
# printing json
data

{'status': 'ok',
 'data': [{'lat': 0.5070677,
   'lon': 101.4477793,
   'uid': 8288,
   'aqi': '74',
   'station': {'name': 'Pekanbaru, Indonesia',
    'time': '2023-08-11T03:00:00+09:00'}},
  {'lat': 22.5367507,
   'lon': 88.3638022,
   'uid': 12746,
   'aqi': '159',
   'station': {'name': 'Ballygunge, Kolkata, India',
    'time': '2023-08-11T01:30:00+09:00'}},
  {'lat': 21.613267,
   'lon': 73.010555,
   'uid': 11843,
   'aqi': '52',
   'station': {'name': 'GIDC, Ankleshwar, India',
    'time': '2023-08-11T02:30:00+09:00'}},
  {'lat': 32.6689873,
   'lon': -94.1674569,
   'uid': 221,
   'aqi': '52',
   'station': {'name': 'Karnack, Tyler, Texas',
    'time': '2023-08-11T01:00:00+09:00'}},
  {'lat': 57.28,
   'lon': 27.07,
   'uid': -212278,
   'aqi': '7',
   'station': {'name': 'Dārziņi', 'time': '2023-08-11T03:26:31+09:00'}},
  {'lat': -38.15588,
   'lon': 146.7842,
   'uid': 12685,
   'aqi': '63',
   'station': {'name': 'Rosedale', 'time': '2023-08-11T03:00:00+09:00'}},
  {'lat': 3

In [6]:
# count number of datapoints
len(data['data'])

1261

In [7]:
# create dataframe
df = pd.DataFrame(data['data'])
df

,lat,lon,uid,aqi,station
0,0.507068,101.447779,8288,74,"{'name': 'Pekanbaru, Indonesia', 'time': '2023..."
1,22.536751,88.363802,12746,159,"{'name': 'Ballygunge, Kolkata, India', 'time':..."
2,21.613267,73.010555,11843,52,"{'name': 'GIDC, Ankleshwar, India', 'time': '2..."
3,32.668987,-94.167457,221,52,"{'name': 'Karnack, Tyler, Texas', 'time': '202..."
4,57.280000,27.070000,-212278,7,"{'name': 'Dārziņi', 'time': '2023-08-11T03:26:..."
...,...,...,...,...,...
1256,35.656700,-88.809200,7497,31,"{'name': 'Jackson, Tennessee, USA', 'time': '2..."
1257,30.217500,-92.051389,6515,33,"{'name': 'Lafayette USGS site, Louisiana, USA'..."
1258,34.282353,-118.436605,-398233,67,"{'name': 'San Fernando MS (8358)', 'time': '20..."
1259,11.030287,106.356310,13659,149,"{'name': 'Tây Ninh/Thị xã Tràng Bảng, Vietnam'..."


In [8]:
# normalise the 'station' column and expand into separate columns
normalised = pd.json_normalize(df['station'])

# concatenate the normalized data with the original dataframe
df = pd.concat([df, normalised], axis=1)

# drop the original 'station' column
df.drop('station', axis=1, inplace=True)

In [9]:
df

,lat,lon,uid,aqi,name,time
0,0.507068,101.447779,8288,74,"Pekanbaru, Indonesia",2023-08-11T03:00:00+09:00
1,22.536751,88.363802,12746,159,"Ballygunge, Kolkata, India",2023-08-11T01:30:00+09:00
2,21.613267,73.010555,11843,52,"GIDC, Ankleshwar, India",2023-08-11T02:30:00+09:00
3,32.668987,-94.167457,221,52,"Karnack, Tyler, Texas",2023-08-11T01:00:00+09:00
4,57.280000,27.070000,-212278,7,Dārziņi,2023-08-11T03:26:31+09:00
...,...,...,...,...,...,...
1256,35.656700,-88.809200,7497,31,"Jackson, Tennessee, USA",2023-08-11T03:00:00+09:00
1257,30.217500,-92.051389,6515,33,"Lafayette USGS site, Louisiana, USA",2023-08-11T02:00:00+09:00
1258,34.282353,-118.436605,-398233,67,San Fernando MS (8358),2023-08-11T02:00:00+09:00
1259,11.030287,106.356310,13659,149,"Tây Ninh/Thị xã Tràng Bảng, Vietnam",2023-08-11T02:00:00+09:00


In [10]:
# split the 'time' column into 'date' and 'time'
df['date'] = df['time'].str.split('T').str[0]
df['time'] = df['time'].str.split('T').str[1].str[:-6]

In [11]:
df

,lat,lon,uid,aqi,name,time,date
0,0.507068,101.447779,8288,74,"Pekanbaru, Indonesia",03:00:00,2023-08-11
1,22.536751,88.363802,12746,159,"Ballygunge, Kolkata, India",01:30:00,2023-08-11
2,21.613267,73.010555,11843,52,"GIDC, Ankleshwar, India",02:30:00,2023-08-11
3,32.668987,-94.167457,221,52,"Karnack, Tyler, Texas",01:00:00,2023-08-11
4,57.280000,27.070000,-212278,7,Dārziņi,03:26:31,2023-08-11
...,...,...,...,...,...,...,...
1256,35.656700,-88.809200,7497,31,"Jackson, Tennessee, USA",03:00:00,2023-08-11
1257,30.217500,-92.051389,6515,33,"Lafayette USGS site, Louisiana, USA",02:00:00,2023-08-11
1258,34.282353,-118.436605,-398233,67,San Fernando MS (8358),02:00:00,2023-08-11
1259,11.030287,106.356310,13659,149,"Tây Ninh/Thị xã Tràng Bảng, Vietnam",02:00:00,2023-08-11


In [12]:
# check datatypes - aqi is currently an object type
df.dtypes

lat     float64
lon     float64
uid       int64
aqi      object
name     object
time     object
date     object
dtype: object

In [13]:
# clean the 'aqi' column by replacing non-numeric values with NaN
df['aqi'] = pd.to_numeric(df['aqi'], errors='coerce')

# drop rows with NaN values in the 'aqi' column
df.dropna(subset=['aqi'], inplace=True)

# convert the 'aqi' column to integer
df['aqi'] = df['aqi'].astype(int)

# check datatypes - aqi is now integer so ready for analysis
df.dtypes

lat     float64
lon     float64
uid       int64
aqi       int64
name     object
time     object
date     object
dtype: object

In [14]:
# final df
df

,lat,lon,uid,aqi,name,time,date
0,0.507068,101.447779,8288,74,"Pekanbaru, Indonesia",03:00:00,2023-08-11
1,22.536751,88.363802,12746,159,"Ballygunge, Kolkata, India",01:30:00,2023-08-11
2,21.613267,73.010555,11843,52,"GIDC, Ankleshwar, India",02:30:00,2023-08-11
3,32.668987,-94.167457,221,52,"Karnack, Tyler, Texas",01:00:00,2023-08-11
4,57.280000,27.070000,-212278,7,Dārziņi,03:26:31,2023-08-11
...,...,...,...,...,...,...,...
1256,35.656700,-88.809200,7497,31,"Jackson, Tennessee, USA",03:00:00,2023-08-11
1257,30.217500,-92.051389,6515,33,"Lafayette USGS site, Louisiana, USA",02:00:00,2023-08-11
1258,34.282353,-118.436605,-398233,67,San Fernando MS (8358),02:00:00,2023-08-11
1259,11.030287,106.356310,13659,149,"Tây Ninh/Thị xã Tràng Bảng, Vietnam",02:00:00,2023-08-11


In [15]:
# export to csv file
df.to_csv('data/air_pollution_data.csv', index=False)

In [16]:
# export to JSON file
with open('data/air_pollution_data.json', 'w') as json_file:
    json.dump(data, json_file, indent=4)

# Exploratory Analysis

In [17]:
# stations with lowest aqi (least polluted)
least_polluted = df.sort_values(by='aqi', ascending = True)
least_polluted.head()

,lat,lon,uid,aqi,name,time,date
991,-30.851700,143.074200,12397,1,Western LLS White Cliffs,20:00:00,2023-08-10
218,56.227334,41.294753,-344119,1,Dobrograd,03:23:27,2023-08-11
346,47.130000,142.766000,-212287,1,Зелёная улица,03:26:01,2023-08-11
98,45.124000,36.712000,-222379,2,Приморский бульвар,03:25:26,2023-08-11
1016,-29.557419,31.120064,14060,2,"Hambanathi Tongaat, eThekwini Metro, South Africa",03:00:00,2023-08-11


In [18]:
# stations with highest aqi (most polluted)
most_polluted = df.sort_values(by='aqi', ascending = False)
most_polluted.head()

,lat,lon,uid,aqi,name,time,date
1146,-2.956974,104.719347,-87088,500,Palembang,03:26:22,2023-08-11
955,51.351281,25.850776,-247483,500,"mikroraion Peremohy, 8",02:03:33,2023-08-11
494,23.530809,55.485926,3773,402,"Al Qua’a, UAE",03:00:00,2023-08-11
397,54.228000,37.632000,-343618,362,улица Гагарина,03:25:46,2023-08-11
497,62.452084,-114.364031,42,339,"Yellowknife, NorthWestTerritories, Canada",00:00:00,2023-08-11
